# Pandas version of the code in "R for Marketing Research and Analytics"

## Ch 3 - Simulating data and plotting

In [1]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print only the last output of the cell
#InteractiveShell.ast_node_interactivity = "last_expr"
# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np

In [3]:
#define constants
k_stores = 20  # 20 stores, using "k_" for "constant"
k_years = 2
k_weeks_per_year = 52   

In [4]:
# create data frame of initially missing values to hold the data
columns=['storeNum', 'Year', 'Week', 'p1sales', 'p2sales', 'p1price','p2price','p1prom','p2prom','country']
df_store = pd.DataFrame(data=np.empty(shape=(k_stores*k_weeks_per_year*k_years, len(columns))),
                        columns=columns)
df_store.tail(5)

,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
2075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_store.shape

(2080, 10)

### Repeating values and sequences
When simulating data, we often need to repeat values in category ("factor") columns.  This section goes through some basic python and pandas for repeating elements or sequences to do this.

In [6]:
##### Sequences ######
# Use python list multiplication to repeat sequences
seq = ['a', 'b', 'c']
print(seq * 6)

# works for integers too if a python list.  pd.Series wouldn't work here
# because the "*6" would be interpreted as an integer math with a vector
# and it would "broadcast" the 1-dimensional number 6 and multiply it by
# all the integers in the Series
seq = [1, 2, 3]
print(seq * 6)
print(pd.Series(seq) * 6)

##### Repeating Values ######
# The numpy repeat() repeats the individual values it finds in the array
np.repeat(seq,6)  # pd.Series(seq).repeat(6) #also works


['a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c']
[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
0     6
1    12
2    18
dtype: int64


array([1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3])

### Add categorical columns
Create 2 series that will represent the store number and country for each observation

In [7]:
store_num = [num for num in range(101,100+k_stores+1)] 
print(store_num)
store_cty = ["US"]*3 + ["DE"]*5 + ["GB"]*3 + ["BR"]*2 + ["JP"]*4 + ["AU"]*1 + ["CN"]*2
print(store_cty)
len(store_cty)  # make sure this is the right length

[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
['US', 'US', 'US', 'DE', 'DE', 'DE', 'DE', 'DE', 'GB', 'GB', 'GB', 'BR', 'BR', 'JP', 'JP', 'JP', 'JP', 'AU', 'CN', 'CN']


20

In [8]:
# now replace the appropriate column in the dataframe with those values
# and repeat the sequences so it is the right number of rows
df_store.storeNum = store_num * k_weeks_per_year * k_years
df_store.country = store_cty * k_weeks_per_year * k_years
#df_store.tail(21)

In [9]:
# Now do the same for Week and Year columns
df_store.Week = [week for week in range(1,k_weeks_per_year+1)] * k_stores*2
# for year, we want each store to have 
one_store_worth_of_year_entries = [[year]*k_weeks_per_year for year in range(1,k_years+1)]
# now flattend this list of lists:
flat_list = [item for sublist in one_store_worth_of_year_entries for item in sublist]
print(flat_list)
df_store.Year = flat_list * k_stores
df_store.tail(21)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
2059,120,2,32,0.0,0.0,0.0,0.0,0.0,0.0,CN
2060,101,2,33,0.0,0.0,0.0,0.0,0.0,0.0,US
2061,102,2,34,0.0,0.0,0.0,0.0,0.0,0.0,US
2062,103,2,35,0.0,0.0,0.0,0.0,0.0,0.0,US
2063,104,2,36,0.0,0.0,0.0,0.0,0.0,0.0,DE
2064,105,2,37,0.0,0.0,0.0,0.0,0.0,0.0,DE
2065,106,2,38,0.0,0.0,0.0,0.0,0.0,0.0,DE
2066,107,2,39,0.0,0.0,0.0,0.0,0.0,0.0,DE
2067,108,2,40,0.0,0.0,0.0,0.0,0.0,0.0,DE
2068,109,2,41,0.0,0.0,0.0,0.0,0.0,0.0,GB


In [10]:
# Check the overall structure
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 10 columns):
storeNum    2080 non-null int64
Year        2080 non-null int64
Week        2080 non-null int64
p1sales     2080 non-null float64
p2sales     2080 non-null float64
p1price     2080 non-null float64
p2price     2080 non-null float64
p1prom      2080 non-null float64
p2prom      2080 non-null float64
country     2080 non-null object
dtypes: float64(6), int64(3), object(1)
memory usage: 162.6+ KB


### Set category variables (called "factors" in R)

In [11]:
# Category Primer
s = pd.Series(pd.Categorical(['grade a', 'd paper', 'd paper', 'grade a', 'grade a', 'b paper']))
s
s.cat.categories
# rename the categories
s.cat.categories = ['good', 'bad', 'very good'] #must correspond to order of existing s.cat.categories
s
# rename/reorder the categories and add missing categories
s = s.cat.set_categories(['very bad', 'bad', 'medium', 'good', 'very good'])
s
# converting a string (object) Series to a categorical
s = pd.Series(['a','b','a','d']).astype('category')
s


0    grade a
1    d paper
2    d paper
3    grade a
4    grade a
5    b paper
dtype: category
Categories (3, object): [b paper, d paper, grade a]

Index(['b paper', 'd paper', 'grade a'], dtype='object')

0    very good
1          bad
2          bad
3    very good
4    very good
5         good
dtype: category
Categories (3, object): [good, bad, very good]

0    very good
1          bad
2          bad
3    very good
4    very good
5         good
dtype: category
Categories (5, object): [very bad, bad, medium, good, very good]

0    a
1    b
2    a
3    d
dtype: category
Categories (3, object): [a, b, d]

In [12]:
# set country as a categorical variable
df_store.country = df_store.country.astype('category')
# set storeeNum as categorical variable
df_store.storeNum = df_store.storeNum.astype('category')

### Simulating data points

In [13]:
# specify the seed to make test results repeatable
np.random.seed(98250)
# randomly select weeks that products p1 and p2 were promoted. The vars specify how often they are promoted
p1_promo_rate = 0.1
p2_promo_rate = 0.15
df_store.p1prom = np.random.binomial(n=1, p=p1_promo_rate, size=df_store.p1prom.shape[0]) # 1 10% of the time, 0 otherwise; repeat num_row times
df_store.p2prom = np.random.binomial(n=1, p=p2_promo_rate, size=df_store.p2prom.shape[0])
df_store.sample(20)

,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
1427,108,2,24,0.0,0.0,0.0,0.0,0,0,DE
831,112,2,52,0.0,0.0,0.0,0.0,0,0,BR
1584,105,1,25,0.0,0.0,0.0,0.0,0,0,DE
1999,120,1,24,0.0,0.0,0.0,0.0,0,1,CN
1043,104,1,4,0.0,0.0,0.0,0.0,0,0,DE
216,117,1,9,0.0,0.0,0.0,0.0,0,0,JP
2017,118,1,42,0.0,0.0,0.0,0.0,0,0,AU
507,108,2,40,0.0,0.0,0.0,0.0,0,0,DE
1670,111,1,7,0.0,0.0,0.0,0.0,0,0,GB
840,101,1,9,0.0,0.0,0.0,0.0,0,0,US


In [14]:
# randomly select one of five prices by sampling with replacement
p1_possible_prices = [2.19, 2.29, 2.49, 2.79, 2.99]
p2_possible_prices = [2.29, 2.49, 2.59, 2.99, 3.19]
df_store.p1price = np.random.choice(p1_possible_prices, size=df_store.p1price.shape[0], replace=True)
df_store.p2price = np.random.choice(p2_possible_prices, size=df_store.p2price.shape[0], replace=True)
df_store.sample(20)

,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
789,110,2,10,0.000000e+00,0.000000e+00,2.29,2.49,1,0,GB
1550,111,2,43,0.000000e+00,0.000000e+00,2.19,2.49,0,1,GB
1709,110,1,46,0.000000e+00,0.000000e+00,2.99,2.49,0,1,GB
1023,104,2,36,0.000000e+00,0.000000e+00,2.79,2.59,0,0,DE
591,112,2,20,0.000000e+00,0.000000e+00,2.19,2.29,0,0,BR
795,116,2,16,0.000000e+00,0.000000e+00,2.29,2.59,0,0,JP
1474,115,1,19,0.000000e+00,0.000000e+00,2.99,2.99,0,0,JP
252,113,1,45,0.000000e+00,0.000000e+00,2.99,2.29,0,0,BR
1382,103,1,31,0.000000e+00,0.000000e+00,2.99,2.49,0,1,US
195,116,2,40,0.000000e+00,0.000000e+00,2.79,2.49,0,0,JP


In [15]:
# Randomly generate sales (unit sales) data.  Use Poisson with lambda=mean weekly sales
p1_mean_weekly_sales = 120
p2_mean_weekly_sales = 100
# first randomly generate sales
tmp_p1sales = np.random.poisson(lam=p1_mean_weekly_sales, size=df_store.p1price.shape[0])
tmp_p2sales = np.random.poisson(lam=p1_mean_weekly_sales, size=df_store.p2price.shape[0])
df_store.head(10)
print(tmp_p1sales[:10])
print(tmp_p2sales[:10])
# now scale these counts up or down according to relative price.  Price effects often
# follow a logarithmic function rather than a linear function, so we use log(price)
# we assume that sales vary as the inverse ratio of the two prices
tmp_p1sales = tmp_p1sales * np.log(df_store.p2price) / np.log(df_store.p1price)
tmp_p2sales = tmp_p2sales * np.log(df_store.p1price) / np.log(df_store.p2price)
print(tmp_p1sales[:10])
print(tmp_p2sales[:10])
# now assume sales get a 30% or 40% lift when each product is promoted
p1_promo_uplift = 0.3
p2_promo_uplift = 0.4
df_store.p1sales = np.floor(tmp_p1sales * (1 + df_store.p1prom*p1_promo_uplift))
df_store.p2sales = np.floor(tmp_p2sales * (1 + df_store.p2prom*p2_promo_uplift))
df_store.sample(20)

,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
0,101,1,1,6.782548e-310,6.782517e-310,2.29,3.19,0,0,US
1,102,1,2,6.782517e-310,6.782517e-310,2.49,3.19,1,1,US
2,103,1,3,6.782529e-310,6.782529e-310,2.79,3.19,0,0,US
3,104,1,4,6.782559e-310,6.782517e-310,2.19,2.49,0,0,DE
4,105,1,5,6.782517e-310,6.782517e-310,2.49,3.19,0,0,DE
5,106,1,6,6.782557e-310,6.782560e-310,2.79,3.19,0,0,DE
6,107,1,7,6.782513e-310,6.782513e-310,2.79,3.19,0,1,DE
7,108,1,8,6.782560e-310,6.782560e-310,2.29,2.59,1,0,DE
8,109,1,9,6.782559e-310,6.782559e-310,2.19,2.29,0,0,GB
9,110,1,10,6.782560e-310,6.782560e-310,2.99,2.49,0,1,GB


[149 127  96 142 108 116 118 116 106 113]
[134 118 111 108 113 109 142 126 106 100]
0    208.608699
1    161.487941
2    108.535569
3    165.255632
4    137.328328
5    131.147145
6    133.408303
7    133.235256
8    112.037657
9     94.120744
dtype: float64
0     95.710294
1     92.799499
2     98.179796
3     92.801678
4     88.867317
5     96.410791
6    125.599379
7    109.700694
8    100.287709
9    120.058549
dtype: float64


,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
228,109,1,21,95.0,136.0,2.49,2.29,0,0,GB
1362,103,1,11,125.0,113.0,2.29,2.29,0,0,US
1698,119,1,35,128.0,100.0,2.19,2.49,0,0,CN
1730,111,2,15,171.0,100.0,2.19,2.99,0,0,GB
1924,105,2,1,133.0,105.0,2.19,2.29,0,0,DE
1319,120,2,20,92.0,125.0,2.79,2.49,0,0,CN
1152,113,1,9,119.0,103.0,2.49,2.59,0,0,BR
575,116,2,4,121.0,121.0,2.19,2.29,0,0,JP
1683,104,1,20,90.0,238.0,2.99,2.49,0,1,DE
1084,105,1,45,126.0,167.0,2.99,2.29,1,0,DE


## Exploring / Plotting Data

In [16]:
print('Count of p1price occurances')
df_store.groupby(['p1price']).count().loc[:,'Year'] # 'Year' could have been any other column (as long as no NaN's in that column for p1price)
print('...showing why the "Year" column used above doesnt matter')
df_store.groupby(['p1price']).count()
print('The count of p1prices by year (year 1 and year 2)')
pd.crosstab(index=df_store.p1price, columns=df_store.Year, margins=True)
print('The mean value of the other variables at each p1price value')
pd.pivot_table(data=df_store, index=df_store.p1price, margins=True, aggfunc=np.mean)

Count of p1price occurances


p1price
2.19    409
2.29    422
2.49    417
2.79    404
2.99    428
Name: Year, dtype: int64

...showing why the "Year" column used above doesnt matter


,storeNum,Year,Week,p1sales,p2sales,p2price,p1prom,p2prom,country
p1price,,,,,,,,,
2.19,409,409,409,409,409,409,409,409,409
2.29,422,422,422,422,422,422,422,422,422
2.49,417,417,417,417,417,417,417,417,417
2.79,404,404,404,404,404,404,404,404,404
2.99,428,428,428,428,428,428,428,428,428


The count of p1prices by year (year 1 and year 2)


Year,1,2,All
p1price,,,
2.19,204,205,409
2.29,203,219,422
2.49,205,212,417
2.79,199,205,404
2.99,229,199,428
All,1040,1040,2080


The mean value of the other variables at each p1price value


,Week,Year,p1prom,p1sales,p2price,p2prom,p2sales
p1price,,,,,,,
2.19,26.479218,1.501222,0.097800,155.117359,2.695134,0.166259,104.046455
2.29,25.488152,1.518957,0.116114,147.497630,2.693318,0.125592,106.827014
2.49,27.568345,1.508393,0.122302,134.585132,2.711103,0.172662,118.738609
2.79,25.797030,1.507426,0.106436,118.646040,2.712277,0.138614,132.844059
2.99,27.140187,1.464953,0.098131,110.544393,2.692336,0.156542,143.822430
All,26.500000,1.500000,0.108173,133.199519,2.700721,0.151923,121.334135


In [28]:
# indexing
# create a boolean vector which describes which rows are store 101
(df_store.storeNum ==101)[:3]  #show just the first 3
# show just the rows for store 101 in year 1
df_store.loc[(df_store.storeNum==101) & (df_store.Year==1),:].iloc[:3,:] #iloc shows just the first 3

# show the mean of each of columns 3 thru 9
df_store.iloc[:,3:9].apply(np.mean, axis=0)

# using the indexing above, show total (sum) sales for store 101 in year 1
df_store.loc[(df_store.storeNum==101) & (df_store.Year==1),:].iloc[:,3:9].apply(np.sum, axis=0)

# now try specifying an anonymous function
df_store.loc[(df_store.storeNum==101) & (df_store.Year==1),:].iloc[:,3:9].apply(lambda x: np.mean(x)-np.median(x), axis=0)


0     True
1    False
2    False
Name: storeNum, dtype: bool

,storeNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
0,101,1,1,208.0,95.0,2.29,3.19,0,0,US
20,101,1,21,177.0,89.0,2.19,3.19,0,0,US
40,101,1,41,111.0,132.0,2.79,2.59,0,0,US


p1sales    133.199519
p2sales    121.334135
p1price      2.551587
p2price      2.700721
p1prom       0.108173
p2prom       0.151923
dtype: float64

p1sales    7053.00
p2sales    5922.00
p1price     129.58
p2price     141.18
p1prom        5.00
p2prom        3.00
dtype: float64

p1sales    2.134615
p2sales    5.884615
p1price    0.001923
p2price    0.125000
p1prom     0.096154
p2prom     0.057692
dtype: float64